In [1]:
import keras
import ascends as asc
import numpy as np
from pprint import pprint
import pandas
import matplotlib.pyplot as plt
%matplotlib inline
import random
import operator
import pandas as pd
from pandas import DataFrame
import numpy as np

from sklearn.decomposition import PCA

Using TensorFlow backend.


In [2]:
csv_file = "data/creep_more_features.csv"
input_col = None
cols_to_remove = ["id", "NAME", "RuptureTime"]
target_col = "LMP"

data_df, x_train, y_train, header_x, header_y =\
asc.data_load_shuffle(csv_file, input_col, cols_to_remove, target_col)

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
def feature_analysis(col_name, only_minmax = False):
    
    array_values = data_df[col_name].values
    final_dict = {}
    final_dict['col_mean'] = np.mean(array_values)
    final_dict['col_stdev'] = np.std(array_values)
    final_dict['col_min'] = np.min(array_values)
    final_dict['col_max'] = np.max(array_values)

    if only_minmax !=True:
        fs_dict, final_report = asc.correlation_analysis_all(data_df, target_col)
        for key in final_report.loc[col_name].keys():
            final_dict[key] = final_report.loc[col_name,key]
            try:
                final_dict[key+"_rank"] = int(final_report.rank(axis=0, ascending=False).loc[col_name, key])
            except:
                final_dict[key+"_rank"] = -1
            
    return final_dict

def show_histogram(col_name):
    ax = data_df[col_name].plot.hist(bins=10, alpha=0.5)

def show_scatter(col_name):
    ax = data_df.plot.scatter(x=col_name,y=target_col)

In [ ]:
col_name = 'xFE'

final_dict = feature_analysis(col_name)
print(final_dict)
show_histogram(col_name)

In [ ]:
show_scatter(col_name)

In [ ]:
def select_data_by_value(data_df, col_name, value):
    data_df_by_value = data_df.loc[data_df[col_name]==value]
    return data_df_by_value

In [ ]:
def conditional_over_threshold(col_list, constraint_list, num_of_data_point_threshold=10, threshold=0.8, criteria='PCC_SQRT'):
    print("* Analyzing features .. ")
    for constraint_key in constraint_list:   
        print("\n", constraint_key, "being processed .. ")
        distinct_vals = sorted(list(set(data_df[constraint_key].values)))
        print("number of distinct values = ", len(distinct_vals))
        if len(distinct_vals)>30:
            for val in distinct_vals:
                data_df_by_value = select_data_by_value(data_df, constraint_key, val)
                for col in col_list:
                    num_of_data_points = data_df_by_value[col].count()
                    if num_of_data_points>num_of_data_point_threshold and col!=constraint_key:
                        fs_dict, final_report = asc.correlation_analysis_all(data_df_by_value, target_col)
                        criteria_val = final_report[criteria].loc[col]

                        if criteria_val>threshold:
                            print("[ "+col+" vs. "+target_col+" ] when", constraint_key.strip(), '=', val, ", '"+criteria+"' = ", final_report[criteria].loc[col],'# of data points=', num_of_data_points)
                            ax = data_df_by_value.plot.scatter(x=col,y=target_col)
                            plt.show()
    print("- Done - ")
        

In [ ]:
feature_analysis_col = [item for item in header_x if item not in ['T2','T1','Stress','dT']]
#conditional_over_threshold(col_list = feature_analysis_col, constraint_list = header_x, \
#                           threshold = 0.9, num_of_data_point_threshold = 10, criteria='PCC_SQRT')

In [ ]:
def random_feature_test(number_of_features = 10, test_limit = 5000, top_k = 10, model_type="LR"):
    
    test_cnt = 0
    test_result = {}

    while True:
        if test_cnt%500==0:
            print(test_cnt, "processed ..")
        test_cnt+=1
        input_col = random.sample(list(header_x), number_of_features)
        data_df_sample, x_train_sample, y_train_sample, header_x_sample, header_y_sample = asc.data_load_shuffle(csv_file, input_col, cols_to_remove, target_col)
        model_parameters = asc.default_model_parameters()
        model = asc.define_model_regression(model_type, model_parameters, x_header_size = x_train_sample.shape[1])
        predictions, actual_values = asc.train_and_predict(model, x_train_sample, y_train_sample, scaler_option='StandardScaler', num_of_folds=5)
        MAE, R2 = asc.evaluate(predictions, actual_values)
        for col in input_col:
            try:
                test_result[col].append(R2) 
            except:
                test_result[col] = [R2]
        if test_cnt==test_limit:
            break
            
    test_result_avg = {}
    for key in test_result.keys():
        test_result_avg[key] = np.mean(test_result[key])

    sorted_x = sorted(test_result_avg.items(), key=operator.itemgetter(1), reverse= True)
    return sorted_x[:top_k]

In [ ]:
def find_similar_features(header_x, threshold=0.9):
    result_dict = {}
    touched = {}
    
    data_df_corr = data_df.corr()
    for index, row in data_df_corr.iterrows():
        for i in header_x:
            corr_val = row[i]
            try:
                touched[i]
            except:
                touched[i] = False
            if touched[i] == False and index>i and corr_val>threshold and index!=i:
                try:
                    result_dict[index].append(i)
                except:
                    result_dict[index] = [i]  
                touched[i] = True
                
    return result_dict

In [ ]:
result_dict = find_similar_features(header_x, threshold=0.95)
pprint(result_dict)

In [ ]:
pca = PCA(n_components=2)
pca.fit_transform(x_train)

x_train_pca = pca.fit_transform(x_train)
x_train_pca_df = pd.DataFrame(data = x_train_pca
             , columns = ['principal component 1', 'principal component 2'])
x_train_pca_df_target = pd.concat([x_train_pca_df, data_df[[target_col]]], axis = 1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
ax.scatter(x_train_pca_df_target[['principal component 1']]
               , x_train_pca_df_target[['principal component 2']], 
           c=x_train_pca_df_target[['LMP']].values, edgecolor='none', alpha=0.9, 
           cmap=plt.cm.get_cmap('coolwarm', 10))
ax.grid()


In [ ]:
plt.close()
x_train_pca

In [ ]:
dim_n = 5
pca = PCA(n_components=dim_n)
pca.fit_transform(x_train)
x_train_pca = pca.fit_transform(x_train)
model_parameters = asc.default_model_parameters()
model = asc.define_model_regression('LR', model_parameters, x_header_size = x_train_pca.shape[1])
predictions, actual_values = asc.train_and_predict(model, x_train_pca, y_train, scaler_option='StandardScaler', num_of_folds=5)
MAE, R2 = asc.evaluate(predictions, actual_values)
print("# of features = ",dim_n, MAE, R2)
        

In [ ]:
# trying to simulate experiments
# generate input and test

scaler_option = 'StandardScaler'
data_df, x_train, y_train, header_x, header_y =\
asc.data_load_shuffle(csv_file, input_col, cols_to_remove, target_col)
model_parameters = asc.default_model_parameters()
model = asc.define_model_regression("LR", model_parameters, x_header_size = x_train.shape[1])
x_train_, scale = asc.rescale_x(scaler_option, x_train)
y_train_ = y_train.reshape(y_train.shape[0],)
model.fit(x_train_, y_train_)

find_key = 'xNB' # 변경시킬 값
key_idx = 0
for i in range(0, len(header_x)):
    if header_x[i]==find_key:
        key_idx = i
        break

final_dict = feature_analysis(find_key)
num_of_test = 10
train_x_to_test = []
idx_to_select = 0 # 어떤 기준점에서 값을 바꿔갈 것인가, 고정할 값들을 결정

selected_x_train = x_train[idx_to_select]
for item in np.linspace(final_dict['col_min'],final_dict['col_max'],num_of_test):
    new_x_train = selected_x_train.copy()
    new_x_train[key_idx] = item
    train_x_to_test.append(new_x_train)
    
train_x_to_test_ = scale.transform(train_x_to_test)
result = model.predict(train_x_to_test_)
print(result)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel(find_key, fontsize = 15)
ax.set_ylabel(target_col, fontsize = 15)
ax.set_title(target_col+' when changing '+find_key, fontsize = 20)
ax.scatter(np.linspace(final_dict['col_min'],final_dict['col_max'],num_of_test)
               , result)
ax.grid()


In [ ]:
# trying to simulate experiments
# generate input and test

# 모든 데이터 포인트를 고려

scaler_option = 'StandardScaler'
data_df, x_train, y_train, header_x, header_y =\
asc.data_load_shuffle(csv_file, input_col, cols_to_remove, target_col)
model_parameters = asc.default_model_parameters()
model = asc.define_model_regression("RF", model_parameters, x_header_size = x_train.shape[1])
x_train_, scale = asc.rescale_x(scaler_option, x_train)
y_train_ = y_train.reshape(y_train.shape[0],)
model.fit(x_train_, y_train_)

find_key = 'xNB' # 변경시킬 값
key_idx = 0
for i in range(0, len(header_x)):
    if header_x[i]==find_key:
        key_idx = i
        break

final_dict = feature_analysis(find_key)
num_of_test = 10
train_x_to_test = []

xs = []
ys = []

for item in np.linspace(final_dict['col_min'],final_dict['col_max'],num_of_test):
    new_x_trains = []
    for x_train_item in x_train:
        new_x_train = x_train_item.copy()
        new_x_train[key_idx] = item
        new_x_trains.append(new_x_train)
        xs.append(item)
    
    new_x_trains_ = scale.transform(new_x_trains)
    result = model.predict(new_x_trains_)
    for i in result:
        ys.append(i)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel(find_key, fontsize = 15)
ax.set_ylabel(target_col, fontsize = 15)
ax.set_title(target_col+' when changing '+find_key, fontsize = 20)
ax.scatter(xs,ys)
ax.grid()


In [ ]:
# trying to simulate experiments
# generate input and test

# 모든 데이터 포인트를 고려

for key in header_x:
    scaler_option = 'StandardScaler'
    data_df, x_train, y_train, header_x, header_y =\
    asc.data_load_shuffle(csv_file, input_col, cols_to_remove, target_col)
    model_parameters = asc.default_model_parameters()
    model = asc.define_model_regression("RF", model_parameters, x_header_size = x_train.shape[1])
    x_train_, scale = asc.rescale_x(scaler_option, x_train)
    y_train_ = y_train.reshape(y_train.shape[0],)
    model.fit(x_train_, y_train_)

    find_key = key # 변경시킬 값
    key_idx = 0
    for i in range(0, len(header_x)):
        if header_x[i]==find_key:
            key_idx = i
            break

    final_dict = feature_analysis(find_key)
    num_of_test = 10
    train_x_to_test = []

    xs = []


    ys = []
    num_of_inputs = 50

    mins = []
    maxs = []
    means = []
    generated_input = []
    for i in range(0, len(header_x)):
        final_dict = feature_analysis(header_x[i], only_minmax = True)
        mins.append(final_dict['col_min'])
        maxs.append(final_dict['col_max'])
        means.append(final_dict['col_mean'])

    x_tests = []
    x_interest_col = []

    for k in range(0, num_of_inputs):
        generated_input = []
        for i in range(0, len(header_x)):
            rand_val = random.uniform(mins[i],maxs[i])
            
            if i==key_idx:
                x_interest_col.append(rand_val)
            else:
                rand_val = means[i]
            generated_input.append(rand_val)
            
        x_tests.append(generated_input)

    x_tests_ = scale.transform(x_tests)
    result = model.predict(x_tests_)

    plt.close()
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel(find_key, fontsize = 15)
    ax.set_ylabel(target_col, fontsize = 15)
    ax.set_title(target_col+' when changing (others mean value)'+find_key, fontsize = 20)
    ax.scatter(x_interest_col, result)
    ax.grid()
    plt.show()
